In [1]:
from transformers import Trainer,get_linear_schedule_with_warmup,RobertaTokenizer,BertForSequenceClassification,BitsAndBytesConfig
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
import psutil
from sklearn.model_selection import train_test_split
from peft import prepare_model_for_kbit_training
from peft import get_peft_model
import time
from collections import defaultdict

In [2]:
def print_trainable_parameters(model):
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable:,}")
    print(f"Total parameters: {total:,}")
    print(f"Percentage of trainable params: {100 * trainable / total:.2f}%")

In [3]:
from transformers import RobertaTokenizerFast
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=15)
print(model.config)
print_trainable_parameters(model)
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Number of available GPUs: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU available.")
device

# Pretrained RoBERTa tokenizer (e.g. roberta-base)
pretrained_tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# Your custom tokenizer trained on OOL tokens
custom_tokenizer = RobertaTokenizerFast.from_pretrained('./tokenizer') # Try to remove Fast?

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 51

In [4]:
pretrained_vocab = set(pretrained_tokenizer.get_vocab().keys())
custom_vocab = set(custom_tokenizer.get_vocab().keys())

# Identify tokens present in custom but not in pretrained
new_tokens = list(custom_vocab - pretrained_vocab)

print(f"Found {len(new_tokens)} new tokens to add.")

Found 28604 new tokens to add.


In [5]:
# Add new tokens to the pretrained tokenizer
num_added = pretrained_tokenizer.add_tokens(new_tokens)
print(f"Successfully added {num_added} new tokens.")

Successfully added 28604 new tokens.


In [6]:
model.resize_token_embeddings(len(pretrained_tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(78869, 768, padding_idx=1)

In [7]:
from peft import get_peft_model, LoraConfig, TaskType

config = LoraConfig(
    task_type=TaskType.SEQ_CLS,     # or SEQ_2_SEQ_LM, TOKEN_CLS, etc.
    r=6,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "value"],  # Adjust for RoBERTa
)

model = get_peft_model(model, config)
print_trainable_parameters(model)
model.to(device)

Trainable parameters: 823,311
Total parameters: 147,448,350
Percentage of trainable params: 0.56%


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(78869, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [8]:
from datasets import load_dataset
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

data = pd.read_pickle('./saved_data/encoded_data.pck')
le = LabelEncoder()
data['target'] = le.fit_transform(data['Attack_type'])

In [9]:
with open("prompt_encoded_data.txt","w") as f:
    for _, row in data.iterrows():
        f.write(f"You are an expert in network traffic classification. Based on the provided network traffic attributes, you must determine whether the traffic is " +
                f"'Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 'SQL_injection', 'Uploading', " + 
                f"'Vulnerability_scanner', or 'XSS'. Here are the encoded attributes, 'encoded_PPFLE: {row['encoded_PPFLE']}'. " + '\n')
#'<s>','<pad>','</s>','<unk>','<mask>'

In [10]:
with open("prompt_encoded_data.txt", "r") as f:
    prompt_lines = f.readlines()

# Strip newline characters
prompt_lines = [prompt_line.strip() for prompt_line in prompt_lines]
prompt_data = {
    "encoded_PPFLE_prompt": prompt_lines, 'Attack_type': data['Attack_type'], 'Attack_label': data['Attack_label'], 'target': data['target']
}
prompt_df = pd.DataFrame(prompt_data)
prompt_df.sample(15)

,encoded_PPFLE_prompt,Attack_type,Attack_label,target
24241,You are an expert in network traffic classific...,SQL_injection,1,11
126612,You are an expert in network traffic classific...,Normal,0,7
36540,You are an expert in network traffic classific...,DDoS_HTTP,1,1
69081,You are an expert in network traffic classific...,Port_Scanning,1,9
128967,You are an expert in network traffic classific...,Normal,0,7
110710,You are an expert in network traffic classific...,Normal,0,7
7036,You are an expert in network traffic classific...,Ransomware,1,10
66093,You are an expert in network traffic classific...,Port_Scanning,1,9
81712,You are an expert in network traffic classific...,Vulnerability_scanner,1,13
6380,You are an expert in network traffic classific...,Ransomware,1,10


In [11]:
train_set = prompt_df.sample(frac=0.7,random_state=42).reset_index(drop=True)

remaining = prompt_df.drop(train_set.index).reset_index(drop=True)

test_set = remaining.sample(frac=0.5,random_state=42).reset_index(drop=True)

val_set = remaining.drop(test_set.index).reset_index(drop=True)

print(train_set.shape,val_set.shape,test_set.shape)

(110460, 4) (23670, 4) (23670, 4)


In [12]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

train_set, test_set = train_test_split(prompt_df, test_size=test_ratio,stratify=prompt_df.iloc[:,-1], random_state=42)
train_set, val_set = train_test_split(train_set, test_size=val_ratio/(val_ratio+train_ratio),stratify=train_set.iloc[:,-1], random_state=42)

In [13]:
train_set.shape,val_set.shape,test_set.shape

((110460, 4), (23670, 4), (23670, 4))

In [14]:
TARGET_LIST = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP',
                'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning',
                'Ransomware', 'SQL_injection', 'Uploading', 'Vulnerability_scanner',
                'XSS']

In [15]:
prompt_df[prompt_df['Attack_type']=='Uploading'].head(5)

,encoded_PPFLE_prompt,Attack_type,Attack_label,target
13140,You are an expert in network traffic classific...,Uploading,1,12
13141,You are an expert in network traffic classific...,Uploading,1,12
13142,You are an expert in network traffic classific...,Uploading,1,12
13143,You are an expert in network traffic classific...,Uploading,1,12
13144,You are an expert in network traffic classific...,Uploading,1,12


class CustomDataset(Dataset):
  def __init__(self,encodings,df,max_len):
    self.encodings = encodings
    self.df = df
    self.max_len=max_len
    self.targets = self.df['target'].tolist()

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    target = self.targets[idx]
    encoding = self.encodings[idx]

    return {
        'input_ids':encoding['input_ids'].flatten(),
        'attention_mask':encoding['attention_mask'].flatten(),
        'targets':torch.tensor(target,dtype=torch.long)
    }

from datasets import load_dataset, Dataset
from transformers import RobertaTokenizerFast
import torch

# Load tokenizer and dataset
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# Define tokenization function
def tokenize_function(row):
    return tokenizer(
        row["encoded_PPFLE_prompt"],               # replace with your actual column
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Tokenize entire dataset
train_set_enc = (Dataset.from_pandas(train_set)).map(tokenize_function, batched=True)
val_set_enc = (Dataset.from_pandas(val_set)).map(tokenize_function, batched=True)
test_set_enc = (Dataset.from_pandas(test_set)).map(tokenize_function, batched=True)

# Save to disk (recommended)
train_set_enc.save_to_disk('vera_prompt_roberta_train_encodings.pt')
val_set_enc.save_to_disk('vera_prompt_roberta_val_encodings.pt')
test_set_enc.save_to_disk('vera_prompt_roberta_test_encodings.pt')

In [16]:
from datasets import load_dataset, Dataset
from datasets import load_from_disk

train_set_enc = load_from_disk('vera_prompt_roberta_train_encodings.pt')
print(len(train_set_enc))
val_set_enc = load_from_disk('vera_prompt_roberta_val_encodings.pt')
print(len(val_set_enc))
test_set_enc = load_from_disk('vera_prompt_roberta_test_encodings.pt')
print(len(test_set_enc))

110460
23670
23670


In [17]:
def set_dataset_format(dataset):
    dataset = dataset.rename_column('target', 'label')
    dataset.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "label"]  # include "label" column
    )
    return dataset

train_dataset = set_dataset_format(train_set_enc)
val_dataset = set_dataset_format(val_set_enc)
test_dataset = set_dataset_format(test_set_enc)

In [18]:
MAX_LEN=512
BATCH_SIZE=32

train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=BATCH_SIZE,
    num_workers=0

)

val_loader = DataLoader(
    val_dataset,
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=0

)

test_loader = DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=0

)

test_data = next(iter(train_loader))

print(test_data['input_ids'].shape)

labels = train_dataset["label"]
print(f"Labels min, max: {labels.min(), labels.max()}")
print(f"Labels dtype: {labels.dtype}")

torch.Size([32, 128])
Labels min, max: (tensor(0), tensor(14))
Labels dtype: torch.int64


for batch in dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["label"].to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

In [19]:
def load_chkpt(model,version):
  return model.load_state_dict(torch.load(f"./saved_model/securityRoBERTa{version}.0.pt",map_location=torch.device('cpu')))

In [ ]:
import numpy as np

def train_model_peft(trainer,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
  trainer.train()

  losses = []
  correct_predictions = 0

  for d in data_loader:
    input_ids = d['input_ids'].to(device)
    attention_mask = d['attention_mask'].to(device)
    labels = d['label'].to(device)
    print("the")
    
    outputs = trainer.model(input_ids,attention_mask, labels=labels)
    logits = outputs.logits  # extract the logits tensor
    _,preds = torch.max(logits,dim=1)
    loss = loss_fn(logits,labels)

    correct_predictions+=torch.sum(preds==labels).cpu()

    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(trainer.model.parameters(),max_norm=1.0)

    optimizer.step()
    scheduler.step()

    optimizer.zero_grad()

  return correct_predictions/n_examples,np.mean(losses)

In [ ]:
def evaluation_model_peft(trainer,data_loader,loss_fn,device,n_examples):
  trainer.train()

  losses = []
  correct_predictions = 0

  for d in data_loader:
    input_ids = d['input_ids'].to(device)
    attention_mask = d['attention_mask'].to(device)
    labels = d['label'].to(device)

    outputs = trainer.model(input_ids,attention_mask, labels=labels)
    logits = outputs.logits  # extract the logits tensor
    _,preds = torch.max(logits,dim=1)
    loss = loss_fn(logits,labels)

    correct_predictions+=torch.sum(preds==labels).cpu()

    losses.append(loss.item())

  return correct_predictions/n_examples,np.mean(losses)

In [22]:
EPOCHS=3
optimizer_prompt_vera = torch.optim.AdamW(model.parameters(),lr=1e-5)
total_steps = len(train_loader)*EPOCHS

scheduler_prompt_vera = get_linear_schedule_with_warmup(
    optimizer_prompt_vera,
    num_warmup_steps= 0,
    num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [23]:
from transformers import TrainingArguments

batch_size = 32
gradient_accumulation_steps = 4

# output dir 
model_version = "securityRoBERTa_BasePromptVeRA_"
model_dir = f"{model_version}"

training_args = TrainingArguments(
    run_name=model_version,
    output_dir=model_dir,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=1,
    lr_scheduler_type="constant",
    logging_dir=f"{model_dir}/logs",
    fp16=True,  # Enable mixed precision training
    dataloader_num_workers=4,  # Adjust based on your CPU capabilities
    gradient_checkpointing=True,  # Enable gradient checkpointing to save memory
    report_to="none",  # Disable reporting to avoid unnecessary overhead
    label_names=TARGET_LIST,
)

In [24]:
from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix
# The parameters after appling LoRA
print_trainable_parameters(model)

# designing computing metrics as per our use case. (F1-Macro is essential and log-loss is optional)
def compute_metrics(p):
    predictions, labels = p.predictions, TARGET_LIST
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(TARGET_LIST, predictions)
    macro_f1 = f1_score(TARGET_LIST, predictions, average='macro')

    return {"accuracy": accuracy, "macro_f1": macro_f1}

model.to(device)

# configure Trainer
trainer_prompt_vera = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

Trainable parameters: 823,311
Total parameters: 147,448,350
Percentage of trainable params: 0.56%


In [25]:
import json

In [26]:
%%time
history_roberta_base_prompt_vera = defaultdict(list)
best_accuracy_roberta_base_prompt_vera=0
print("1")

# Record start time
start_time = time.time()

for epoch in tqdm(range(EPOCHS)):
  print(f"Epoch {epoch+1}/{EPOCHS}")
  train_acc_roberta_base_prompt_vera,train_loss_roberta_base_prompt_vera = train_model_peft(trainer_prompt_vera,train_loader,loss_fn,optimizer_prompt_vera,device,scheduler_prompt_vera,len(train_set))
  val_acc_roberta_base_prompt_vera,val_loss_roberta_base_prompt_vera = evaluation_model_peft(trainer_prompt_vera,val_loader,loss_fn,device,len(val_set))
  history_roberta_base_prompt_vera['train_acc'].append(train_acc_roberta_base_prompt_vera)
  history_roberta_base_prompt_vera['train_loss'].append(train_loss_roberta_base_prompt_vera)
  history_roberta_base_prompt_vera['val_acc'].append(val_acc_roberta_base_prompt_vera)
  history_roberta_base_prompt_vera['val_loss'].append(val_loss_roberta_base_prompt_vera)
  print(f"Train Loss {train_loss_roberta_base_prompt_vera} | Validation Loss {val_loss_roberta_base_prompt_vera} | Training Accuracy {train_acc_roberta_base_prompt_vera} | Validation Accuracy {val_acc_roberta_base_prompt_vera}")

  if val_acc_roberta_base_prompt_vera>best_accuracy_roberta_base_prompt_vera:
    trainer.save_model(f"./saved_model/{model_version}{epoch+1}.0.pt")
    best_accuracy_roberta_base_prompt_vera = val_acc_roberta_base_prompt_vera

# Record end time
end_time = time.time()

# Calculate training time
history_roberta_base_prompt_vera['training_time'].append(end_time - start_time)

# Convert to regular dict and save as JSON
with open("./saved_model/history_roberta_base_prompt_vera.txt", "w") as f:
    json.dump(history_roberta_base_prompt_vera, f)

1


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/3


Epoch,Training Loss,Validation Loss
1,2.608800,No log


the


TypeError: max() received an invalid combination of arguments - got (SequenceClassifierOutput, dim=int), but expected one of:
 * (Tensor input, *, Tensor out = None)
 * (Tensor input, Tensor other, *, Tensor out = None)
 * (Tensor input, int dim, bool keepdim = False, *, tuple of Tensors out = None)
 * (Tensor input, name dim, bool keepdim = False, *, tuple of Tensors out = None)


In [27]:
# Load JSON and convert back to defaultdict
with open("./saved_model/history_roberta_base_prompt_vera.txt", "r") as f:
    history_roberta_base_prompt_vera_json = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)